## Predicting Airlines Stocks based on Oil Prices

### I. Problem Statement

As jet fuel price is the largest cost in airlines, it can affect on their profit and as a result on their shares price in the stock exchange. If the price of crude oil climbed to \$100 a barrel, for example, jet fuel would surpass \$3 per gallon. A passenger flying coast to coast in the U.S. will pay \$50 extra. For cargo carriers the implication could be loss of market share to ground carriers and a sharp decline in profits. Therefore, airline industry is very sensitive to oil price fluctuations.

In this experiment, in addition to examining this relationship contemporaneously, we also investigate whether any predictive powers are present between oil prices and airline stocks

#### 1. Include all required libraries

In [44]:
library(sqldf)
library(caret)
library(gbm)
library(ggplot2)

#### 2. Read in all datasets

In [ ]:
setwd('/Users/birupakhya/Documents/Projects/SDMproject/')
#reading file and setting the junks values to NA
jetfuel <- read.csv(file = 'jetfuelprices.csv', skip = 12, header = T, as.is = T, na.strings = "#N/A")
delta <- read.csv(file = 'Deltastocks.csv', skip = 0, header = T, as.is = T, na.strings = "NA")
sp500 <- read.csv(file = 'Proj-SP500.csv', skip = 0, header = T, as.is = T, na.strings = "NA")
brentoil <- read.csv(file = 'brentoilprices.csv', skip = 12, header = T, as.is = T, na.strings = c("NA", "#N/A" ))

### II. Data Understanding & Quality Check

#### 1. Check Missing values

In [ ]:
#checking the NA values
colSums(is.na(jetfuel))
colSums(is.na(delta))
colSums(is.na(sp500))
colSums(is.na(brentoil))

There are 91 and 90 missing values in the Jet Fuel and Brent Oil datasets. We will take care of them in a later section. Next, let us do some time series analysis of the prices of all variables

### III. Data Preparation

#### 1. Ensure variables have the correct classes

In [ ]:
#setting up the dates format and extracting required columns
sp500$Date <- as.Date(sp500$Date, '%Y-%m-%d')
sp500Adj <- sp500[,c('Date','Adj.Close')]
colnames(sp500Adj)[2] <- 'sp500Adj.Close'

delta$Date <- as.Date(delta$Date, '%Y-%m-%d')
deltaAdj <- delta[,c('Date','Adj.Close')]
colnames(deltaAdj)[2] <- 'DeltaAdj.Close'

jetfuel$DATE <- as.Date(jetfuel$DATE, '%Y-%m-%d')
colnames(jetfuel)[2] <- 'JetFuelValue'
brentoil$DATE <- as.Date(brentoil$DATE, '%Y-%m-%d')
colnames(brentoil)[2] <- 'BrentOilValue'

#### 2. Impute missing values

In [ ]:
#converting the NAs to mean values of the column - Imputation
jetfuel[is.na(jetfuel[,2]), 2] <- mean(jetfuel[,2], na.rm = TRUE)
brentoil[is.na(jetfuel[,2]), 2] <- mean(brentoil[,2], na.rm = TRUE)

#### 3. Select data for common dates

In [ ]:
finaldata <- sqldf('SELECT *
                   FROM sp500Adj 
                                 INNER JOIN deltaAdj ON (sp500Adj.Date = deltaAdj.Date)
                                 INNER JOIN brentoil ON (sp500Adj.Date = brentoil.DATE)
                                 INNER JOIN jetfuel ON (sp500Adj.Date = jetfuel.DATE)')

#### 4. Create dummy variables

In [ ]:
# compute price change for snp500 data
snp_pc1 = c(); snp_pc2= c(); snp_pc3 = c(); snp_pc4 = c(); snp_pc5 = c();
for (i in 7:nrow(finaldata))
  {
  snp_pc1[i] <- (finaldata$sp500Adj.Close[i - 1] - finaldata$sp500Adj.Close[i - 2])/finaldata$sp500Adj.Close[i - 2]
  snp_pc2[i] <- (finaldata$sp500Adj.Close[i - 2] - finaldata$sp500Adj.Close[i - 3])/finaldata$sp500Adj.Close[i - 3]
  snp_pc3[i] <- (finaldata$sp500Adj.Close[i - 3] - finaldata$sp500Adj.Close[i - 4])/(finaldata$sp500Adj.Close[i - 4])
  snp_pc4[i] <- (finaldata$sp500Adj.Close[i - 4] - finaldata$sp500Adj.Close[i - 5])/(finaldata$sp500Adj.Close[i - 5])
  snp_pc5[i] <- (finaldata$sp500Adj.Close[i - 5] - finaldata$sp500Adj.Close[i - 6])/(finaldata$sp500Adj.Close[i - 6])
  }

# compute price change for brent oil
brent_pc1 = c(); brent_pc2= c(); brent_pc3 = c(); brent_pc4 = c(); brent_pc5 = c();
for (i in 7:nrow(finaldata))
{
  brent_pc1[i] <- (finaldata$BrentOilValue[i - 1] - finaldata$BrentOilValue[i - 2])/(finaldata$BrentOilValue[i - 2])
  brent_pc2[i] <- (finaldata$BrentOilValue[i - 2] - finaldata$BrentOilValue[i - 3])/(finaldata$BrentOilValue[i - 3])
  brent_pc3[i] <- (finaldata$BrentOilValue[i - 3] - finaldata$BrentOilValue[i - 4])/(finaldata$BrentOilValue[i - 4])
  brent_pc4[i] <- (finaldata$BrentOilValue[i - 4] - finaldata$BrentOilValue[i - 5])/(finaldata$BrentOilValue[i - 5])
  brent_pc5[i] <- (finaldata$BrentOilValue[i - 5] - finaldata$BrentOilValue[i - 6])/(finaldata$BrentOilValue[i - 6])
}


# compute price change for jet fuel oil
jetfuel_pc1 = c(); jetfuel_pc2= c(); jetfuel_pc3 = c(); jetfuel_pc4 = c(); jetfuel_pc5 = c();
for (i in 7:nrow(finaldata))
{
  jetfuel_pc1[i] <- (finaldata$JetFuelValue[i - 1] - finaldata$JetFuelValue[i - 2])/(finaldata$JetFuelValue[i - 2])
  jetfuel_pc2[i] <- (finaldata$JetFuelValue[i - 2] - finaldata$JetFuelValue[i - 3])/(finaldata$JetFuelValue[i - 3])
  jetfuel_pc3[i] <- (finaldata$JetFuelValue[i - 3] - finaldata$JetFuelValue[i - 4])/(finaldata$JetFuelValue[i - 4])
  jetfuel_pc4[i] <- (finaldata$JetFuelValue[i - 4] - finaldata$JetFuelValue[i - 5])/(finaldata$JetFuelValue[i - 5])
  jetfuel_pc5[i] <- (finaldata$JetFuelValue[i - 5] - finaldata$JetFuelValue[i - 6])/(finaldata$JetFuelValue[i - 6])
}

# compute price change for Delta adjusted close
delta_pc1 = c(); delta_pc2= c(); delta_pc3 = c(); delta_pc4 = c(); delta_pc5 = c();
for (i in 7:nrow(finaldata))
{
  delta_pc1[i] <- (finaldata$DeltaAdj.Close[i - 1] - finaldata$DeltaAdj.Close[i - 2])/(finaldata$DeltaAdj.Close[i - 2])
}

#### 5. Merge dataframes

In [ ]:
#created dataset
workdata <- data.frame(snp_pc1, snp_pc2, snp_pc3, snp_pc4, snp_pc5, brent_pc1, brent_pc2, brent_pc3, brent_pc4, brent_pc5, jetfuel_pc1, jetfuel_pc2, jetfuel_pc3, jetfuel_pc4, jetfuel_pc5, delta_pc1 )
workdata <- workdata[7:nrow(workdata),]

#### 6. Create partitions

In [ ]:
#partition
rnum <- (runif(1, .60, .70))
part <-sample(1:nrow(workdata), rnum * nrow(workdata))
trng <- workdata[part,]
test <- workdata[-part,]

## IV. Modelling & Evaluation

#### 1. Create a Regression Model

In [ ]:
#perform regression
linear.reg <- lm(delta_pc1 ~  ., data = trng)
summary(linear.reg);
#root mean square error
linear.rmse  <- sqrt(mean(linear.reg$residuals)^2);
#predicting the test data
linear.predict <- predict(linear.reg,test)

#### 2. Modelling using Logistic Regression

#### 2.1 Simple Logistic Regression

In [ ]:
avg <- mean(workdata$delta_pc1)
workdata$delta_01 <- ifelse(workdata$delta_pc1 >= avg, 1, 0)

#partition
rnum <- (runif(1, .60, .70))
part <-sample(1:nrow(workdata), rnum * nrow(workdata))
trng <- workdata[part,]
test <- workdata[-part,]

lr.delta <- glm(delta_01 ~ . -delta_pc1, family=binomial(link="logit"), data=trng)
summary(lr.delta)

#### 2.2. Stepwise Logistic Regression

In [ ]:
#iterations of glm by removing the non significant columns one by one, starting with least significant
lr.delta.01 <- glm(delta_01 ~ snp_pc1 + brent_pc1 + brent_pc2 + brent_pc3 + jetfuel_pc1 + jetfuel_pc3 + jetfuel_pc4, family=binomial(link="logit"), data=trng)
summary(lr.delta.01)

lr.delta.02 <- glm(delta_01 ~ snp_pc1 + brent_pc1 + brent_pc3 + jetfuel_pc1, family=binomial(link="logit"), data=trng)
summary(lr.delta.02)

In [ ]:
#Let's predict
pred1 <- predict(lr.delta , newdata=test, type="response")
pred2 <- predict(lr.delta.01 , newdata=test, type="response")
pred3 <- predict(lr.delta.02 , newdata=test, type="response")

pred_v <- c(pred1,pred2,pred3)

# #converting predictions > 50% to 1 and remaining to 0
pred_1 <- ifelse(pred1 > 0.5,1,0)
pred_2 <- ifelse(pred2 > 0.5,1,0)
pred_3 <- ifelse(pred3 > 0.5,1,0)

# #converting predictions > 50% to 1 and remaining to 0
# pred_1 <- ifelse(pred1 > avg,1,0)
# pred_2 <- ifelse(pred2 > avg,1,0)
# pred_3 <- ifelse(pred3 > avg,1,0)

Comparison of model accuracy of different steps.

In [ ]:
#measuring accuracy Accuracy : 0.6752, 0.6907, 0.6874
confusionMatrix(test$delta_01, pred_1)
confusionMatrix(test$delta_01, pred_2)
confusionMatrix(test$delta_01, pred_3)

#### 3. Modelling using Recursive Partition Tree

In [ ]:
library('rpart')
model.rpt <- rpart(delta_01 ~ snp_pc1 + snp_pc4 + snp_pc5 + brent_pc1 + brent_pc2 + brent_pc4 + jetfuel_pc5 + jetfuel_pc3 + jetfuel_pc1, data=trng, cp=0)
#plot(model.rpt)
#text(model.rpt, use.n= T, digits=3, cex=0.6)
prediction.rpt <- predict(model.rpt, newdata = test, type="vector")
pred_rpt <- ifelse(prediction.rpt > 0.5,1,0)
# printcp(model.rpt)
confusionMatrix(test$delta_01, pred_rpt)

#### 4. Modelling using Gradient Boosting

In [ ]:
model.gbm <- gbm(delta_01 ~ snp_pc1 + brent_pc1 + brent_pc2 + brent_pc3 + jetfuel_pc1 + jetfuel_pc3 + jetfuel_pc4, data=trng , n.trees=5000, interaction.depth =6, shrinkage=0.01)
prediction.gbm <- predict(model.gbm, newdata = test, n.trees=5000, type="response")
# head(prediction.gbm[])
# tail(prediction.gbm[])
# summary(prediction.gbm)

pred_gbm <- ifelse(prediction.gbm > 0.42,1,0)
# printcp(model.rpt)
# table(pred_gbm, test$delta_01)

# accuracy remains around 61-62%
confusionMatrix(pred_gbm, test$delta_01)